<a href="https://colab.research.google.com/github/ryyhan/RandomCodes/blob/main/llmComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28
!pip install groq
!pip install boto3

In [ ]:
import time
import openai
from groq import Groq
import boto3
import json
from botocore.exceptions import ClientError
import pandas as pd

In [ ]:
# Mock functions to simulate LLM calls
def call_gpt4o_mini(prompt):
  openai.api_key = ''
  response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )

  return (len(response['choices'][0]['message']['content']))


In [ ]:
def call_llama3_1_groq(prompt):
  client = Groq(api_key="",)


  completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=1,
    stop=None,
)
  return len(completion.choices[0].message.content)

In [ ]:
def call_llama3_1_aws(prompt):
  aws_region = ''
  access_key_id = ''
  secret_access_key = ''

  # Create a Bedrock client
  bedrock = boto3.client(
    'bedrock-runtime',
    region_name=aws_region,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key
  )

  model_id = "meta.llama3-1-8b-instruct-v1:0"


  native_request = {
    "prompt": prompt
  }

  request = json.dumps(native_request)


  try:
    response = bedrock.invoke_model(modelId=model_id, body=request)
    response_body = json.loads(response.get('body').read())


  except ClientError as e:
    # Handle any errors that occur during the invocation
    print(f"An error occurred: {e.response['Error']['Message']}")

  return len(response_body.get('generation'))

In [ ]:
def compare_llms(prompts):
    results = []

    for prompt in prompts:
        result = {}
        result['Prompt'] = prompt

        # Measuring GPT-4o Mini
        start_time = time.time()
        character_count = call_gpt4o_mini(prompt)
        elapsed_time = time.time() - start_time
        result['GPT-4o Mini Characters'] = character_count
        result['GPT-4o Mini Time'] = elapsed_time
        result['GPT-4o Mini Characters/Second'] = character_count / elapsed_time if elapsed_time > 0 else 0

        # Measuring LLaMA 3.1 from Groq
        start_time = time.time()
        character_count = call_llama3_1_groq(prompt)
        elapsed_time = time.time() - start_time
        result['LLaMA 3.1 (Groq) Characters'] = character_count
        result['LLaMA 3.1 (Groq) Time'] = elapsed_time
        result['LLaMA 3.1 (Groq) Characters/Second'] = character_count / elapsed_time if elapsed_time > 0 else 0

        # Measuring LLaMA 3.1 from AWS Bedrock
        start_time = time.time()
        character_count = call_llama3_1_aws(prompt)
        elapsed_time = time.time() - start_time
        result['LLaMA 3.1 (AWS) Characters'] = character_count
        result['LLaMA 3.1 (AWS) Time'] = elapsed_time
        result['LLaMA 3.1 (AWS) Characters/Second'] = character_count / elapsed_time if elapsed_time > 0 else 0

        results.append(result)

    return pd.DataFrame(results)

In [ ]:
prompts = [
    "What are the advantages of using renewable energy?",
    "How do solar panels work?",
    "What is wind energy?",
    "Explain geothermal energy and its uses.",
    "What are the benefits of electric vehicles?"
]

# Compare results for the defined prompts
comparison_results = compare_llms(prompts)
comparison_results

,Prompt,GPT-4o Mini Characters,GPT-4o Mini Time,GPT-4o Mini Characters/Second,LLaMA 3.1 (Groq) Characters,LLaMA 3.1 (Groq) Time,LLaMA 3.1 (Groq) Characters/Second,LLaMA 3.1 (AWS) Characters,LLaMA 3.1 (AWS) Time,LLaMA 3.1 (AWS) Characters/Second
0,What are the advantages of using renewable ene...,2979,5.010935,594.499877,2017,19.384260,104.053493,2839,5.694711,498.532722
1,How do solar panels work?,1744,3.511463,496.659129,2375,0.834551,2845.842142,2518,5.617745,448.222539
2,What is wind energy?,1783,2.995769,595.172721,2026,61.761554,32.803578,2694,5.680483,474.255438
3,Explain geothermal energy and its uses.,3388,8.380569,404.268517,2964,1.048769,2826.170499,1700,5.641884,301.317769
4,What are the benefits of electric vehicles?,2723,5.449988,499.634094,2134,0.751632,2839.156326,2528,5.743744,440.131063


In [ ]:
comparison_results[["GPT-4o Mini Time", "LLaMA 3.1 (Groq) Time", "LLaMA 3.1 (AWS) Time"]]

,GPT-4o Mini Time,LLaMA 3.1 (Groq) Time,LLaMA 3.1 (AWS) Time
0,5.010935,19.384260,5.694711
1,3.511463,0.834551,5.617745
2,2.995769,61.761554,5.680483
3,8.380569,1.048769,5.641884
4,5.449988,0.751632,5.743744


In [ ]:
comparison_results[["GPT-4o Mini Characters/Second", "LLaMA 3.1 (Groq) Characters/Second", "LLaMA 3.1 (AWS) Characters/Second"]]

,GPT-4o Mini Characters/Second,LLaMA 3.1 (Groq) Characters/Second,LLaMA 3.1 (AWS) Characters/Second
0,594.499877,104.053493,498.532722
1,496.659129,2845.842142,448.222539
2,595.172721,32.803578,474.255438
3,404.268517,2826.170499,301.317769
4,499.634094,2839.156326,440.131063
